In [2]:
!pip install sentencepiece protobuf datasets transformers trl textstat peft bitsandbytes nltk --quiet
!pip install -U bitsandbytes accelerate --quiet

v4 is about oncology notes

## import

In [3]:
# Standard library imports
import csv
import json
import os
import random
import re
import sys
import time
from collections import Counter
from collections import defaultdict
cache_dir = "/mnt/c/Users/yc/.cache/huggingface"
os.environ['HF_HOME'] = cache_dir
# Third-party data and ML libraries
import pandas as pd
import torch
import nltk
from nltk.tokenize import word_tokenize

# Hugging Face ecosystem
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)

# Fine-tuning and optimization
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Text analysis and readability metrics
from textstat import (
    flesch_kincaid_grade, 
    flesch_reading_ease,
    smog_index, 
    gunning_fog, 
    dale_chall_readability_score,
    text_standard, 
    syllable_count
)

# Optional: Uncomment if needed
from huggingface_hub import login
login(token=hftoken)  # Move token to environment variable

/home/yc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'hftoken' is not defined

## data

In [6]:
breastca_unannotated = pd.read_csv('/mnt/c/Users/yc/Downloads/coral/unannotated/data/breastca_unannotated.csv')
pdac_unannotated = pd.read_csv('/mnt/c/Users/yc/Downloads/coral/unannotated/data/pdac_unannotated.csv')



In [11]:
note_text=(breastca_unannotated['note_text'].iloc[0])

print("=" * 50)
sentences = note_text.split('. ')
for i, sentence in enumerate(sentences, 1):
    if sentence.strip():
        sentence = sentence.strip()
        if not sentence.endswith('.') and i < len(sentences):
            sentence += '.'
        print(f"{i:2d}. {sentence}")

print("\n" + "=" * 50 + "\n")

 1. Medical Oncology Consult Note    Patient Name: ***** *****   Patient MRN:  *****   Patient DOB:  11/23/1963   Date of Visit:  12/30/2019  Provider:  ***** ***** *****  Primary Care Provider:  None Per Patient Provider  Referring MD:        Reason for visit:   Chief Complaint   Patient presents with    New Patient Evaluation       Diagnosis:    1.
 2. Malignant neoplasm of overlapping sites of right breast in female, estrogen receptor positive (CMS code)  NM Whole Body Bone Scan    MR Brain with and without Contrast    Complete Blood Count with Differential    Comprehensive Metabolic Panel (BMP, AST, ALT, T.BILI, ALKP, TP, ALB)    Cancer Antigen 15-3    Carcinoembryonic Antigen    Activated Partial Thromboplastin Time    Prothrombin Time    Ambulatory Referral to Integrative Medicine       History of Present Illness:   56 year old female diagnosed in May 2013 with a multifocal Stage IIA right breast cancer.
 3. She had a mastectomy with sentinel node and implant reconstruction in J

In [ ]:

def calculate_readability_scores(text, term=None):
    """Calculate readability scores for a piece of text, optionally removing a term"""
    scoring_text = text
    
    # Remove the term and its variations before scoring if provided
    if term:
        # Create variations of the term to remove (capitalized, lowercase)
        term_variations = [term, term.lower(), term.capitalize()]
        
        for variation in term_variations:
            scoring_text = scoring_text.replace(variation, "")
        
        # Clean up any double spaces created
        scoring_text = " ".join(scoring_text.split())
    
    fk_grade = flesch_kincaid_grade(scoring_text)
    readability = flesch_reading_ease(scoring_text)
    return {
        "fk_grade": fk_grade,
        "readability": readability,
    } 
def txt_to_dict(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines) - 1, 2):
            key = lines[i].strip()    # Odd line are key
            value = lines[i + 1].strip()  # Even line are value
            data_dict[key] = value

    return data_dict


txt_file_path = 'formaldef.txt' 
formaldic = txt_to_dict(txt_file_path)
len(formaldic)

meddict={}
for k,v in formaldic.items():
    meddict[k.split('Listen to pronunciation')[0].split('(')[0]]=v